In [1]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 5)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

epoch [4/200], loss:-1.0450
x:  tensor([[ 12.8644,   3.5891,  -7.2631, -23.0276,  16.2817]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 12.4835,   3.5839,  -7.0596, -23.4237,  17.1383]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 15.3084,   1.4928,  -8.5580, -27.5704,  18.5316]],
       grad_fn=<AddmmBackward>)
epoch [5/200], loss:-1.0675
x:  tensor([[ 17.1714,   1.9273,  16.8825, -27.9099,  -8.8583]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 16.8783,   1.9261,  17.1605, -28.3982,  -8.3237]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 20.3039,  -0.2451,  19.5499, -33.9307, -10.1139]],
       grad_fn=<AddmmBackward>)
epoch [6/200], loss:-1.2554
x:  tensor([[ 37.4270,   3.2455,   5.5854, -35.4136,   3.2416]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 36.9728,   3.1643,   5.9023, -36.1678,   1.6519]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 42.1594,   1.0925,   6.6413, -42.2537,   2.1338]],
       grad_fn=<AddmmBackward>)
epoch [7/200], loss:-1.3362
x:  tensor([[28.0241,

b: tensor([[-27.3184, -13.4902,   7.9594,  18.5553,  -0.6904]],
       grad_fn=<AddmmBackward>)
epoch [31/200], loss:-1.6574
x:  tensor([[-16.1166,  -0.3069,  19.4693,   7.5409,   4.3261]],
       grad_fn=<AddmmBackward>)
y: tensor([[-14.7628,  -0.4911,  19.3665,   7.1362,   4.4632]],
       grad_fn=<AddmmBackward>)
b: tensor([[-11.2705,  -2.9782,  18.8805,   3.6675,   2.9514]],
       grad_fn=<AddmmBackward>)
epoch [32/200], loss:-1.5096
x:  tensor([[-10.8599,  15.2761, -29.6310,   2.7418,  15.7355]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -9.4398,  15.2715, -29.6346,   2.2244,  15.9023]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -4.9018,  13.9321, -31.8719,  -0.6220,  14.5546]],
       grad_fn=<AddmmBackward>)
epoch [33/200], loss:-1.5697
x:  tensor([[ -5.0093,  -8.5226,   2.0760, -11.1028,   2.8646]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -3.5783,  -8.6276,   2.1433, -11.6359,   3.0883]],
       grad_fn=<AddmmBackward>)
b: tensor([[  0.6441, -11.0640,   1.4094, -15.

b: tensor([[-17.0662,  -4.9592,  10.8019,   1.1544,  11.1329]],
       grad_fn=<AddmmBackward>)
epoch [57/200], loss:-1.5763
x:  tensor([[-14.0886,   7.1916,   0.1033,   5.8599,  24.0740]],
       grad_fn=<AddmmBackward>)
y: tensor([[-14.1221,   7.5211,   1.0392,   6.1753,  23.8289]],
       grad_fn=<AddmmBackward>)
b: tensor([[-11.0118,   5.3804,   1.5284,   2.3752,  20.8516]],
       grad_fn=<AddmmBackward>)
epoch [58/200], loss:-1.8035
x:  tensor([[-9.1764,  2.5907,  4.8551, -7.0670,  7.3693]],
       grad_fn=<AddmmBackward>)
y: tensor([[-9.2190,  3.0067,  5.8767, -6.7986,  7.0253]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -7.4079,   1.0434,   5.5535, -11.0303,   5.4211]],
       grad_fn=<AddmmBackward>)
epoch [59/200], loss:-1.6174
x:  tensor([[-19.7593,   5.7562,   1.6380, -19.5852,  -1.8482]],
       grad_fn=<AddmmBackward>)
y: tensor([[-18.9423,   5.3977,   0.8585, -19.5275,  -1.5825]],
       grad_fn=<AddmmBackward>)
b: tensor([[-16.8439,   3.4744,   1.4449, -20.1996,  -2.

b: tensor([[ -4.0935,  -7.5885,  10.0143,  -6.7230, -11.4897]],
       grad_fn=<AddmmBackward>)
epoch [84/200], loss:-1.5235
x:  tensor([[ -3.0857, -15.1398,   6.3933,   0.6031,   5.7432]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -3.2738, -15.5934,   5.4504,   1.5286,   5.6588]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -0.4637, -14.5890,   5.3537,  -1.0787,   2.9276]],
       grad_fn=<AddmmBackward>)
epoch [85/200], loss:-1.3618
x:  tensor([[  4.6237, -13.4613,   0.8105,  -4.5184,  10.3556]],
       grad_fn=<AddmmBackward>)
y: tensor([[  4.6224, -13.1233,   1.5585,  -4.2998,  10.0121]],
       grad_fn=<AddmmBackward>)
b: tensor([[  6.4612, -12.9979,   2.4112,  -7.6232,   6.9862]],
       grad_fn=<AddmmBackward>)
epoch [86/200], loss:-1.3737
x:  tensor([[-19.5606,  -5.2926,  -7.1719,  -8.8750,  10.8066]],
       grad_fn=<AddmmBackward>)
y: tensor([[-19.6946,  -5.4595,  -6.9876,  -8.9020,  11.5739]],
       grad_fn=<AddmmBackward>)
b: tensor([[-14.9804,  -4.9199,  -5.2564, -10.

b: tensor([[-8.2227, -1.3578, -2.2963,  0.5238, 13.6801]],
       grad_fn=<AddmmBackward>)
epoch [111/200], loss:-1.4290
x:  tensor([[ 4.5704,  5.4818, -5.8977,  8.9818, 22.4319]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 3.5958,  5.4854, -6.2164, 10.0485, 21.6227]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 4.2335,  3.8343, -3.8571,  4.8811, 15.5794]],
       grad_fn=<AddmmBackward>)
epoch [112/200], loss:-1.4163
x:  tensor([[ 7.8637,  2.9135, 12.3818,  4.0239,  5.6423]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 8.0919,  3.3741, 12.7876,  3.2391,  5.0398]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 9.0842,  0.9711, 11.9451, -0.7335,  2.0916]],
       grad_fn=<AddmmBackward>)
epoch [113/200], loss:-1.5904
x:  tensor([[ 1.6575, -2.4702, 10.2705, -2.0171, 28.5150]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 1.7988, -2.1371, 10.5348, -2.6576, 27.9887]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 2.7526, -3.5557,  9.3737, -5.7721, 21.8106]],
       grad_fn=<AddmmBackward

b: tensor([[  5.1958,   5.8575,  -1.3035, -11.7248,   1.3516]],
       grad_fn=<AddmmBackward>)
epoch [138/200], loss:-1.4591
x:  tensor([[-5.4306,  7.5598, -6.0604,  6.8089, -0.7867]],
       grad_fn=<AddmmBackward>)
y: tensor([[-6.2563,  7.6403, -6.4013,  7.6122, -1.6205]],
       grad_fn=<AddmmBackward>)
b: tensor([[-3.8862,  4.5527, -3.8499,  4.1653, -2.9749]],
       grad_fn=<AddmmBackward>)
epoch [139/200], loss:-1.3970
x:  tensor([[17.7466, 14.4138,  6.1431, -1.0337, -2.3949]],
       grad_fn=<AddmmBackward>)
y: tensor([[16.7491, 14.4047,  5.3172, -2.4355, -2.2525]],
       grad_fn=<AddmmBackward>)
b: tensor([[15.9595,  9.5507,  5.5653, -3.4774, -2.8240]],
       grad_fn=<AddmmBackward>)
epoch [140/200], loss:-1.4729
x:  tensor([[15.2491,  9.0324,  5.3977,  4.9712, 15.3118]],
       grad_fn=<AddmmBackward>)
y: tensor([[15.2069,  9.4043,  6.1352,  5.2494, 14.6616]],
       grad_fn=<AddmmBackward>)
b: tensor([[13.1237,  5.4631,  5.4638,  0.8182, 10.4286]],
       grad_fn=<AddmmBac

b: tensor([[ 7.8592, -7.4007,  8.4118,  9.3331,  3.5262]],
       grad_fn=<AddmmBackward>)
epoch [165/200], loss:-1.4752
x:  tensor([[ 2.6892, -7.0743,  6.4325, -9.7569,  7.5544]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 2.5270, -7.2097,  6.7582, -9.4516,  8.2618]],
       grad_fn=<AddmmBackward>)
b: tensor([[  2.7572,  -6.8168,   5.2649, -10.1382,   3.1923]],
       grad_fn=<AddmmBackward>)
epoch [166/200], loss:-1.6198
x:  tensor([[ 8.9394, -1.2342,  8.3259,  1.9511, 10.5218]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 8.8430, -0.8662,  9.0385,  2.2528,  9.9138]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 6.6202, -2.0244,  7.6295, -1.3081,  6.1161]],
       grad_fn=<AddmmBackward>)
epoch [167/200], loss:-1.4754
x:  tensor([[21.9826, -1.1867,  5.1300,  6.3109,  4.9369]],
       grad_fn=<AddmmBackward>)
y: tensor([[22.1672, -0.8253,  5.4468,  5.6387,  4.4564]],
       grad_fn=<AddmmBackward>)
b: tensor([[16.9751, -2.5108,  4.6509,  1.5300,  2.5101]],
       grad_fn=<AddmmBac

epoch [192/200], loss:-1.3239
x:  tensor([[-2.3385, -4.5080, -0.0653, -0.1451,  5.9578]],
       grad_fn=<AddmmBackward>)
y: tensor([[-2.5613, -4.9070, -1.0732,  0.3849,  5.8493]],
       grad_fn=<AddmmBackward>)
b: tensor([[-1.8854, -4.7842,  0.0337, -1.7164,  2.0629]],
       grad_fn=<AddmmBackward>)
epoch [193/200], loss:-1.6211
x:  tensor([[ 0.0467,  1.9664,  1.1092,  6.6637, 10.4951]],
       grad_fn=<AddmmBackward>)
y: tensor([[-0.1268,  1.7019,  1.3873,  7.4654, 11.4038]],
       grad_fn=<AddmmBackward>)
b: tensor([[1.6916, 0.1012, 1.6851, 3.6604, 6.2448]], grad_fn=<AddmmBackward>)
epoch [194/200], loss:-1.5570
x:  tensor([[-0.3682,  3.6103,  5.4634,  2.3531,  6.1570]],
       grad_fn=<AddmmBackward>)
y: tensor([[-0.2516,  3.9419,  5.8070,  1.6756,  5.7425]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 0.0290,  1.6946,  4.9791, -1.2830,  3.1230]],
       grad_fn=<AddmmBackward>)
epoch [195/200], loss:-1.6640
x:  tensor([[ 8.5517,  5.0066, -0.3785, -2.8373, 10.2003]],
       gra

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [4]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [2]:
import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
length  600


In [6]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time
print(np.shape(uframes[0]))


#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
print("Length of Class data: ", len(class_data))
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

print("Start")
for i in range(2):
    z = cv2.imread('Images/0c.png')
    z  = cv2.resize(z,(100,100))
    img = z
    print(np.shape(img))
    #ms = Image.fromarray(z)
    #ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    #print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
    


ModuleNotFoundError: No module named 'termcolor'

In [54]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   2

In [55]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [53]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [2]:
img = np.transpose(data1[5],(1,2,0))
print(np.shape(np.transpose(data1[5],(1,2,0))))
cv2.imshow('test', img)

img = data2[0]
ms = Image.fromarray(img)
ms.show()

(100, 100, 3)


TypeError: Cannot handle this data type: (1, 1, 100), <f4